## FantaSanremo 2023

Social Network, Content, and Trend analysis

Il Festival di Sanremo è sempre più social. E così, anche aziende e content creator seguono gli avvenimenti del festival per restare aggiornato su possibili trend da sfruttare e possibilità per crescere. Negli ultimi anni, il "FantaSanremo" ha spopolato tra giovani e meno giovani.

Si tratta di un gioco simile al classico e ormai popolare fantacalcio, ma incentrato proprio sul festival. Ogni partecipante sceglie una squadra composta da 5 cantanti con un massimo di 100 'bauli' (prezzo di acquisto), e - in base ai punti raccolti dalla propria squadra - compete con amici e altri in un campionato a punteggio. Per ottenere punti e riconoscimenti, i cantanti devono svolgere alcune attività presenti nel regolamento di ogni anno, senza dimenticare il festival vero e proprio, e quindi il piazzamento in classifica.

L'interesse nello svolgere questo progetto è quindi raccolto nelle seguenti domande:

- Quali sono i trend relativi a Sanremo 2023 e al FantaSanremo?
- Come possono essere sfruttati dall'azienda X e/o dall'artista Y per promuoversi (a livello di brand)
- Qual è il sentiment generale in merito al tema?
- Ѐ possibile individuare community?

Il festival è "sfruttato" dai brand per farsi conoscere e/o fidelizzare i (possibili) clienti, e dagli artisti per aumentare la propria fanbase o la propria popolarità, oltre ovviamente a competere con la propria canzone per raggiungere il successo.

Gli artisti, in particolare, guadagnano sempre più grazie agli streaming delle varie piattaforme musicali, guadagni che aumentano con il numero di ascolti delle proprie canzoni.

In [ ]:
# imports

import numpy as np
import pandas as pd

import re
import string
from datetime import datetime
import time

from tqdm.notebook import tqdm
import json

#### Data

In [ ]:
fantasanremo23 = pd.read_csv('../data/raw/fantasanremo2023_scraping.csv')

In [ ]:
print(fantasanremo23.shape)
print()
print(fantasanremo23.columns)
print()

fantasanremo23.head()

Il dataset, ottenuto tramite web scraping grazie a "snscraper", è composto da poco più di 28000 righe e 6 colonne.

Il periodo di riferimento, invece, va dal 25/12/2022 al 12/02/2023. Il 25 dicembre corrisponde al giorno precedente all'annuncio dei 'bauli' (valore) necessari per acquistare ciascun cantante nella propria squadra; mentre il 12 febbraio corrisponde al giorno successivo dell'annuncio della classifica finale del FantaSanremo.

#### Data Exploration

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.text import OffsetFrom
import matplotlib.ticker as ticker

import seaborn as sns

In [ ]:
fantasanremo23 = fantasanremo23[['date', 'id', 'content', 'username', 'like_count', 'retweet_count']]

In [ ]:
fantasanremo23.info()

In [ ]:
# remove duplicates
fantasanremo23.drop_duplicates(subset ="id", inplace = True)
fantasanremo23.reset_index(drop = True, inplace = True)
fantasanremo23.shape

In [ ]:
fantasanremo23['date'] = pd.to_datetime(fantasanremo23['date'])

In [ ]:
fantasanremo23.dtypes

In [ ]:
fantasanremo23.isnull().sum()

In [ ]:
print('min date', fantasanremo23.date.min())
print('max date', fantasanremo23.date.max())

In [ ]:
# change date format
day = fantasanremo23['date'].dt.day
month = fantasanremo23['date'].dt.month
year = fantasanremo23['date'].dt.year

date = year.astype(str) + month.astype(str).str.zfill(2) + day.astype(str).str.zfill(2)
date = pd.to_datetime(date, format='%Y%m%d')
fantasanremo23.drop(columns = ['date'], inplace = True)
fantasanremo23['date'] = date

# reorder columns
cols = fantasanremo23.columns.tolist()
cols = cols[-1:] + cols[:-1]
fantasanremo23 = fantasanremo23[cols].copy()

print('Tweet per day:')
print()
print(fantasanremo23.groupby('date').count()['id'].sort_values(ascending=False))

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5), facecolor='#EFE9E6')
ax.set_facecolor('#EFE9E6')

plt.plot(fantasanremo23.groupby('date').id.nunique(), color='#5A5A5A')

# annotation with data coordinates and offset points
ax.annotate(
    xy = (140, 250),
    xycoords = 'figure pixels',
    xytext = (10, 15),
    textcoords = 'offset pixels',
    text = '27 December 2022\nRegistration Openings',
    size = 6,
    color = "grey",
    arrowprops = dict(
        arrowstyle="->", shrinkA=0, shrinkB=5, color="grey", linewidth=0.75,
        connectionstyle="angle3,angleA=50,angleB=-30"
    ) # arrow to connect annotation
)

# annotation with data coordinates and offset points
ax.annotate(
    xy = (740, 440),
    xycoords = 'figure pixels',
    xytext = (15, 15),
    textcoords = 'offset pixels',
    text = '7 February 2023\nStart of the Show',
    size = 6,
    color = "grey",
    arrowprops = dict(
        arrowstyle="->", shrinkA=0, shrinkB=5, color="grey", linewidth=0.75,
        connectionstyle="angle3,angleA=50,angleB=-30"
    ) # arrow to connect annotation
)

# spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# format x axis dates
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%m\n%Y'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=5))

ax.yaxis.grid(linestyle='dashed', alpha=0.5)
ax.set_xlabel('Date')
ax.set_ylabel('Number of Tweets')
ax.set_title('FantaSanremo2023 - Number of Tweets per Date')

plt.savefig('../figures/numTweetsDate.png', dpi=300)
plt.show()

Come possiamo osservare dalla distribuzione di tweet nei vari giorni, i giorni in cui sono stati pubblicati la maggior parte di tweet corrispondono ai giorni del festival o a giorni di annunci particolari, il 2022-12-27 è, ad esempio, il giorno di apertura delle iscrizioni per il FantaSanremo.

In [ ]:
# create a DataFrame with the authors of the tweets and their respective frequency
freq_authors = fantasanremo23['username'].value_counts()
freq_authors.head()

Approfondiamo ora, più nel dettaglio, i tweet risalenti al 27-12-2022.

In [ ]:
# filter data
fantasanremo23_271222 = fantasanremo23[fantasanremo23.date == '2022-12-27'].copy()
fantasanremo23_271222.head()

In [ ]:
# number of unique tweets
fantasanremo23_271222.id.nunique()

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.scatterplot(
    data=fantasanremo23_271222,
    x='retweet_count',
    y='like_count',
    s=200
)

for i, point in fantasanremo23_271222[
    (fantasanremo23_271222.like_count > 1000) | (fantasanremo23_271222.retweet_count > 1000)].iterrows():
    ax.text(point.retweet_count*1.03, point.like_count*1, point.username + ': ' + point.content)

ax.set_xlabel('Number of Retweets')
ax.set_ylabel('Number of Likes')
plt.savefig('../figures/scatterLikeRT.png', dpi=300)
plt.show()

#### Pre-Processing

In [ ]:
import nltk

nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')

from nltk import FreqDist
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer

from bs4 import BeautifulSoup

In [ ]:
fantasanremo23['new_content'] = fantasanremo23['content'].astype(str)

# lower case
fantasanremo23['new_content'] = fantasanremo23['new_content'].str.lower()

# remove punctuation but keep hashtags
fantasanremo23['new_content'] = fantasanremo23['new_content'].str.replace(r'^\w#\s','', regex=True)

# remove stopwords
stop = stopwords.words('italian')
fantasanremo23['new_content'] = fantasanremo23['new_content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

# lemmatization
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

fantasanremo23["new_content"] = fantasanremo23["new_content"].apply(lambda text: lemmatize_words(text))

# remove links
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

fantasanremo23["new_content"] = fantasanremo23["new_content"].apply(lambda text: remove_urls(text))

# remove html
def remove_html(text):
    return BeautifulSoup(text, "lxml").text

fantasanremo23["new_content"] = fantasanremo23["new_content"].apply(lambda text: remove_html(text))

In [ ]:
fantasanremo23['new_content'].head()

In [ ]:
from wordcloud import WordCloud

In [ ]:
# hashtags
text = ' '.join(tweet for tweet in fantasanremo23.new_content)

# extract hashtags
hashtags = re.findall('#(\w+)', text)

ax = pd.DataFrame(hashtags, columns=['hashtag']).value_counts().head(20).plot(kind='barh', figsize=(3,5))
ax.invert_yaxis()

plt.savefig('../figures/hashtags.png', dpi=300, bbox_inches = "tight")
plt.show()

In [ ]:
# generate a wordcloud for the combined text
hashtags = ' '.join(hashtag for hashtag in hashtags)

# setup, generate and save the word cloud image to a file
wc = WordCloud(
    width=1000,
    height=500,
    random_state=0,
    max_font_size=110,
    background_color='white',
    collocations=False).generate(hashtags)

wc.to_file("../figures/WordCloud_hashtags.png")

# show the wordcloud as output
plt.figure()
plt.imshow(wc)
plt.axis("off")
plt.show()

Stesso procedimento, stavolta escludendo hashtag superflui o non interessanti.

In [ ]:
# hashtags
text2 = ' '.join(tweet for tweet in fantasanremo23.new_content)

# extract hashtags
hashtags2 = re.findall('#(\w+)', text2)

hashtags2 = [
    hashtag.strip() for hashtag in hashtags2 if hashtag not in
    [
        'fantasanremo2023',
        'fantasanremo',
        'sanremo2023',
        'sanremo',
        'sanremo23',
        'festivaldisanremo',
        'festivaldisanremo2023',
        'fantasanremo23'
    ]
]

ax = pd.DataFrame(hashtags2, columns=['hashtag']).value_counts().head(20).plot(kind='barh', figsize=(3,5))
ax.invert_yaxis()

plt.savefig('../figures/hashtags2.png', dpi=300, bbox_inches = "tight")

plt.show()

In [ ]:
# generate a wordcloud for the combined text
hashtags2 = ' '.join(hashtag for hashtag in hashtags2)

# setup, generate and save the word cloud image to a file
wc = WordCloud(
    width=1000,
    height=500,
    random_state=0,
    max_font_size=110,
    background_color='white',
    collocations=False).generate(hashtags2)

wc.to_file("../figures/WordCloud_hashtags2.png")

# show the wordcloud as output
plt.figure()
plt.imshow(wc)
plt.axis("off")
plt.show()

In [ ]:
fantasanremo23.to_csv('../data/fantasanremo23.csv')

#### Social Content Analysis

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
fantasanremo23 = pd.read_csv('../data/fantasanremo23.csv', index_col=0)
fantasanremo23.head(1)

In [ ]:
fantasanremo23_sa = fantasanremo23.copy()

_Sentiment Analysis_

twitter-XLM-roBERTa-base for Sentiment Analysis

This is a multilingual XLM-roBERTa-base model trained on ~198M tweets and finetuned for sentiment analysis. The sentiment fine-tuning was done on 8 languages (Ar, En, Fr, De, Hi, It, Sp, Pt) but it can be used for more languages.

https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment

In [ ]:
# preprocess text (username and link placeholder)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax

MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)

model.save_pretrained(MODEL)
tokenizer.save_pretrained(MODEL)

In [ ]:
fantasanremo23_sa['scores'] = fantasanremo23_sa.content.apply(lambda x: softmax(model(**tokenizer(preprocess(x), return_tensors='pt'))[0][0].detach().numpy()))

In [ ]:
fantasanremo23_sa[['content', 'scores']].head()

In [ ]:
fantasanremo23_sa['negative'] = fantasanremo23_sa.scores.apply(lambda x: x[0])
fantasanremo23_sa['neutral'] = fantasanremo23_sa.scores.apply(lambda x: x[1])
fantasanremo23_sa['positive'] = fantasanremo23_sa.scores.apply(lambda x: x[2])

In [ ]:
fantasanremo23_sa.to_csv('../data/fantasanremo23_sa.csv')

#### Social Network Analysis

In [ ]:
fantasanremo23_sn = fantasanremo23.copy()

In [ ]:
# extract user mentions
fantasanremo23_sn['user_mentions'] = fantasanremo23_sn['content'].str.extract('@(\S+)')

In [ ]:
fantasanremo23_sn['user_mentions']

In [ ]:
# lower case
fantasanremo23_sn['user_mentions'] = fantasanremo23_sn['user_mentions'].str.lower()

# define a string of punctuation characters to remove
punctuation = string.punctuation

# Use str.rstrip to remove punctuation from the end of the "text" column
fantasanremo23_sn['user_mentions'] = fantasanremo23_sn['user_mentions'].str.rstrip(punctuation)

In [ ]:
fantasanremo23_sn.to_csv('../data/fantasanremo23_sn.csv')

In [ ]:
author_mentions = fantasanremo23_sn[['username', 'user_mentions']]
author_mentions.head()

In [ ]:
author_mentions.info()

#### Key Takeaways

- Gli hashtags più utilizzati tra i tweet relativi al #FantaSanremo 2023 riguardano i cantanti in gara